# Given a potential suite of simulations, figure out emulator space covered

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import copy
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import camb_cosmo
import fit_linP
import sim_params_cosmo
import sim_params_space
import read_genic
import write_config
import latin_hypercube
import corner
import p1d_arxiv

### Plot initial Latin hypercube

In [2]:
# get list of simulation parameters
add_slope=True
add_running=False
add_mu_He=True
add_mu_H=True
param_space=sim_params_space.SimulationParameterSpace(add_slope=add_slope,add_running=add_running,
                                                      add_mu_He=add_mu_He,add_mu_H=add_mu_H, z_star=4., kp_Mpc=2.)
params=param_space.params
for key,param in params.items():
    print(key,param)

Om_star {'ip': 0, 'min_val': 0.977, 'max_val': 0.986, 'z_star': 4.0, 'latex': '$\\Omega_\\star$'}
Delta2_star {'ip': 1, 'min_val': 0.35, 'max_val': 0.55, 'z_star': 4.0, 'kp_Mpc': 2.0, 'latex': '$\\Delta^2_\\star$'}
Alpha {'ip': 2, 'min_val': 0.0, 'max_val': 0.1, 'latex': '$\\alpha$'}
Beta {'ip': 3, 'min_val': 0.0, 'max_val': 10.0, 'latex': '$\\beta$'}
Gamma {'ip': 4, 'min_val': -10.0, 'max_val': 0.0, 'latex': '$\\gamma$'}
n_star {'ip': 5, 'min_val': -2.53, 'max_val': -2.43, 'z_star': 4.0, 'kp_Mpc': 2.0, 'latex': '$n_\\star$'}
mu_He {'ip': 6, 'min_val': 0.5, 'max_val': 2.0, 'latex': '$\\mu_{\\rm He}$'}
mu_H {'ip': 7, 'min_val': 0.5, 'max_val': 2.0, 'latex': '$\\mu_{\\rm H}$'}


In [3]:
# get pivot point
z_star=params['Om_star']['z_star']
kp_Mpc=params['Delta2_star']['kp_Mpc']
print('z_star =',z_star)
print('kp_Mpc =',kp_Mpc)
Npar=len(params)
param_limits=np.empty([Npar,2])
for key,param in params.items():
    ip=param['ip']
    param_limits[ip][0]=param['min_val']
    param_limits[ip][1]=param['max_val']

z_star = 4.0
kp_Mpc = 2.0


In [ ]:
# generate Latin hypercube 
nsamples=35
seed=123
cube=latin_hypercube.get_hypercube_samples(param_limits, nsamples, prior_points = None, seed=seed)

In [ ]:
labels=['']*Npar
for key,param in params.items():
    ip=param['ip']
    labels[ip]=param['latex']
corner.corner(cube,labels=labels)

###  Plot cosmological parameters in initial setup

In [4]:
# get fiducial cosmology
cosmo_fid = camb_cosmo.get_cosmology()
camb_cosmo.print_info(cosmo_fid)
# setup fiducial linear power model
linP_model_fid=fit_linP.LinearPowerModel(cosmo=cosmo_fid,z_star=4.,k_units='Mpc',kp=2.)
print('fiducial linear power parameters',linP_model_fid.get_params())

H0 = 6.7000E+01, Omega_b h^2 = 2.2000E-02, Omega_c h^2 = 1.2000E-01, Omega_k = 0.0000E+00, Omega_nu h^2 = 0.0000E+00, T_CMB = 2.7255E+00, A_s = 2.1000E-09, n_s = 9.6500E-01, alpha_s = 0.0000E+00
fiducial linear power parameters {'f_star': 0.9898224493399295, 'g_star': 0.9835091539526485, 'Delta2_star': 0.43847556546253746, 'n_star': -2.482734250814445, 'alpha_star': -0.13475631230743798}


In [5]:
linP_model_fid.cosmo.InitPower.pivot_scalar


0.05

In [6]:
#Get input cosmology for output parameters
for Om_star in [0.977, 0.986]:
    for Delta2_star in [0.35, 0.55]:
        for n_star in [-2.53, -2.43]:
            print(Om_star, Delta2_star, n_star)
            simulation_output_parameters = np.array([Om_star, Delta2_star, 0., 1., 0., n_star, 1., 1.])
            cosmo_sim = sim_params_cosmo.cosmo_from_sim_params(params, simulation_output_parameters, linP_model_fid, verbose=True)
            As_kp_Mpc = cosmo_sim.InitPower.As * ((kp_Mpc / cosmo_sim.InitPower.pivot_scalar) ** (cosmo_sim.InitPower.ns - 1))
            print(As_kp_Mpc)


0.977 0.35 -2.53
Omega_m_star = 0.977
Omega_m = 0.2536344755970922
h = 0.7482384635678215
delta_lnA_star = -0.22314068918873717
delta_n_star = -0.04726530191462741
delta_alpha_star = 0.0
delta_lnAs = -0.04878468806337602
delta_ns = -0.04726530191462741
delta_nrun = 0.0
As = 2.0000109522421007e-09
ns = 0.9177346980853726
nrun = 0.0
H0 = 7.4824E+01, Omega_bc = 2.5363E-01, A_s = 2.0000E-09, n_s = 9.1773E-01, alpha_s = 0.0000E+00
1.4765169096315937e-09
0.977 0.35 -2.43
Omega_m_star = 0.977
Omega_m = 0.2536344755970922
h = 0.7482384635678215
delta_lnA_star = -0.22314068918873717
delta_n_star = 0.052734698085372234
delta_alpha_star = 0.0
delta_lnAs = -0.41767263347476835
delta_ns = 0.052734698085372234
delta_nrun = 0.0
As = 1.3830133578695658e-09
ns = 1.017734698085372
nrun = 0.0
H0 = 7.4824E+01, Omega_bc = 2.5363E-01, A_s = 1.3830E-09, n_s = 1.0177E+00, alpha_s = 0.0000E+00
1.47651690963159e-09
0.977 0.55 -2.53
Omega_m_star = 0.977
Omega_m = 0.2536344755970922
h = 0.7482384635678215
delta_l

In [ ]:
cosmo_sim.InitPower.As


 - For each point in cube, write configuration files to run simulation

In [ ]:
zs=np.arange(1.8, 5.6, 0.2)
print('Redshifts =', zs)
sims_dir='test_cube/'
#os.makedirs(sims_dir,exist_ok=True)
#write_config.write_cube_json_file(sims_dir,params,cube)
info_cosmologies=[]
for sample in range(nsamples):
    sim_params=cube[sample]
    print(sample,sim_params)
    cosmo_sim=sim_params_cosmo.cosmo_from_sim_params(params,sim_params,linP_model_fid,verbose=False)
    sim_dir=sims_dir+'/sim_pair_'+str(sample)+'/'
    #os.makedirs(sim_dir,exist_ok=True)
    # construct linear power model and store in JSON format
    linP_model_sim=fit_linP.LinearPowerModel(cosmo_sim,z_star=z_star,k_units='Mpc',kp=kp_Mpc)
    #write_config.write_sim_json_file(sim_dir,params,sim_params,linP_model_sim,zs=zs)
    # store information for plots
    info_cosmo={'cosmo':cosmo_sim,'sim_params':sim_params,'linP_params':linP_model_sim.get_params()}
    info_cosmologies.append(info_cosmo)

In [ ]:
print(cosmo_sim)


In [ ]:
print('linP_params',info_cosmologies[0]['linP_params'])
print('sim_params',info_cosmologies[0]['sim_params'])

In [ ]:
# plot corner plot for derived parameters
derived_labels=[r'$H_0$',r'$\Omega_m$',r'$A_s$']
if add_slope:
    derived_labels.append(r'$n_s$')
if add_running:
    derived_labels.append(r'$\alpha_s$')
Npar=len(derived_labels)
Ncos=len(info_cosmologies)
print(derived_labels)
derived_params=np.ndarray([Ncos,Npar])
for icosmo in range(Ncos):
    cosmo = info_cosmologies[icosmo]['cosmo']
    H0=cosmo.H0
    h=H0/100.0
    omh2=cosmo.omch2+cosmo.ombh2
    derived_params[icosmo][0]=H0
    derived_params[icosmo][1]=(omh2/h**2)
    derived_params[icosmo][2]=cosmo.InitPower.As
    if add_slope:
        derived_params[icosmo][3]=cosmo.InitPower.ns
    if add_running:
        derived_params[icosmo][4]=cosmo.InitPower.nrun
corner.corner(derived_params,labels=derived_labels)

In [ ]:
arxiv=p1d_arxiv.ArxivP1D(basedir=sims_dir,skewers_label=None,verbose=True)
Nmodels=len(arxiv.data)
print('# models =',Nmodels)

In [ ]:
labels=[r'$\Delta_p^2$',r'$n_p$',r'$\alpha_p$',r'$f_p$']
Npar=len(labels)
database=np.empty([Nmodels,Npar])
database[:,0]=arxiv.Delta2_p
database[:,1]=arxiv.n_p
database[:,2]=arxiv.alpha_p
database[:,3]=arxiv.f_p

In [ ]:
corner.corner(database, labels=labels,plot_contours=False)

In [ ]:
def plot_parameter_pair(arxiv,param_1,param_2,plot_z=True):
    # figure out values of param_1,param_2 in arxiv
    val_1=np.array([arxiv.data[i][param_1] for i in range(len(arxiv.data))])
    print(param_1,'first values in arxiv',val_1[:5])
    val_2=np.array([arxiv.data[i][param_2] for i in range(len(arxiv.data))])
    print(param_2,'first values in arxiv',val_2[:5])
    if plot_z:
        z=np.array([arxiv.data[i]['z'] for i in range(len(arxiv.data))])
        zmin=min(z)
        zmax=max(z)
        plt.scatter(val_1,val_2,c=z,s=5,vmin=zmin, vmax=zmax)
        cbar=plt.colorbar()
        cbar.set_label("Redshift", labelpad=+1)
    else:
        plt.scatter(val_1,val_2,s=5)
    plt.xlabel(param_1)
    plt.ylabel(param_2)

In [ ]:
plot_parameter_pair(arxiv,'Delta2_p','f_p')

In [ ]:
plot_parameter_pair(arxiv,'n_p','alpha_p')